In [2]:
import requests
import pandas as pd
import io
import time
import datetime
import numpy as np
import json

In [3]:
TWSE_list = pd.DataFrame()
def Crawler_TWSE_List():
  global TWSE_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
  page = requests.get(url)
  TWSE_list = pd.read_html(page.text)[0]
  TWSE_list.columns = TWSE_list.iloc[0]
  end = 0
  for i in range(0,len(TWSE_list)):
    if (TWSE_list.at[i, '有價證券代號及名稱'][0:4] == '上市認購'):
        end = i
  TWSE_list = TWSE_list.iloc[2:end]
  TWSE_list = TWSE_list[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)']]
  TWSE_list  = TWSE_list.rename(columns={'有價證券代號及名稱': 'Stockcode', '國際證券辨識號碼(ISIN Code)': 'Name'})
  TWSE_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TWSE_list)):
    TWSE_list.at[i, 'Name'] = TWSE_list.at[i, 'Stockcode'][5:]
    TWSE_list.at[i, 'Stockcode'] = TWSE_list.at[i, 'Stockcode'][0:4]

In [4]:
TPEx_list = pd.DataFrame()
def Crawler_TPEx_List():
  global TPEx_list
  url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=4'
  page = requests.get(url)
  TPEx_list = pd.read_html(page.text)[0]
  TPEx_list.columns = TPEx_list.iloc[0]
  start = 0
  end = 0
  for i in range(0,len(TPEx_list)):
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '股票'):
        start = i+1
    if (TPEx_list.at[i, '有價證券代號及名稱'][0:4] == '特別股'):
        end = i
  TPEx_list = TPEx_list.iloc[start:end]
  TPEx_list = TPEx_list[['有價證券代號及名稱', '國際證券辨識號碼(ISIN Code)']]
  TPEx_list = TPEx_list.rename(columns={'有價證券代號及名稱': 'Stockcode', '國際證券辨識號碼(ISIN Code)': 'Name'})
  TPEx_list.reset_index(drop=True, inplace=True)
  for i in range (0,len(TPEx_list)):
    TPEx_list.at[i, 'Name'] = TPEx_list.at[i, 'Stockcode'][5:]
    TPEx_list.at[i, 'Stockcode'] = TPEx_list.at[i, 'Stockcode'][0:4]

In [5]:
StockPrice_TWSE = pd.read_csv('Stock_TWSE.csv', header=[0,1], index_col=0)
StockPrice_TPEx = pd.read_csv('Stock_TPEx.csv', header=[0,1], index_col=0)

In [6]:
different_Tpex = None
different_TWSE = None
def Update_Stock_List():
  global different_Tpex
  global different_TWSE
  Crawler_TPEx_List()
  Crawler_TWSE_List()
  if list(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:])) == list(TPEx_list.iloc[:,0]):
    print('TPEx Pass')
  else:
    if (len(TPEx_list) > (len(dict.fromkeys(StockPrice_TPEx.columns.get_level_values(0)[1:]))-1)):
        different_Tpex = set(TPEx_list.iloc[:,0]) - set(StockPrice_TPEx.columns.get_level_values(0)[1:])
        different_Tpex = list(different_Tpex)
        print(different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Close'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Volume'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Foundation'), None)
            StockPrice_TPEx.insert(len(StockPrice_TPEx.columns), (different_Tpex[i], 'Yield'), None)
            StockPrice_TPEx.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TPEx.pop(('Date',       'Date'))
            StockPrice_TPEx.insert(0, ('Date',       'Date'), temp)
    else:
        different_Tpex = set(StockPrice_TPEx.columns.get_level_values(0)[1:]) - set(TPEx_list.iloc[:,0])
        different_Tpex = list(different_Tpex)
        print(different_Tpex)
        for i in range(0,len(different_Tpex)):
            StockPrice_TPEx.drop(columns=[different_Tpex[i]], inplace=True)
            
  if list(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:])) == list(TWSE_list.iloc[:,0]):
    print('TWSE Pass')
  else:
    if (len(TWSE_list) > (len(dict.fromkeys(StockPrice_TWSE.columns.get_level_values(0)[1:]))-1)):
        different_TWSE = set(TWSE_list.iloc[:,0]) - set(StockPrice_TWSE.columns.get_level_values(0)[1:])
        different_TWSE = list(different_TWSE)
        print(different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Close'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Volume'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Foundation'), None)
            StockPrice_TWSE.insert(len(StockPrice_TWSE.columns), (different_TWSE[i], 'Yield'), None)
            StockPrice_TWSE.sort_index(axis = 1, level=0, inplace=True, sort_remaining = False)
            temp = StockPrice_TWSE.pop(('Date',       'Date'))
            StockPrice_TWSE.insert(0, ('Date',       'Date'), temp)
    else:
        different_TWSE = set(StockPrice_TWSE.columns.get_level_values(0)[1:]) - set(TWSE_list.iloc[:,0])
        different_TWSE = list(different_TWSE)
        print(different_TWSE)
        for i in range(0,len(different_TWSE)):
            StockPrice_TWSE.drop(columns=[different_TWSE[i]], inplace=True)

In [7]:
def crawler(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+ datetime +'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    for i in range(0,len(test_df)):
        try:
            if (len(test_df.loc[i,'代號']) != 4):
                test_df.drop(i,inplace=True)
        except TypeError:
            if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                test_df.drop(i,inplace=True)
    test_df = test_df[['代號', '收盤 ', '成交股數  ']]
    return test_df

def crawler_Foundation(datetime):
    url = 'https://www.tpex.org.tw/web/stock/3insti/daily_trade/3itrade_hedge_result.php?l=zh-tw&o=csv&se=EW&t=D&d='+ datetime +'&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
        if '代號' in text:
            initial_point = i
            break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if len(test_df) > 1:
        for i in range(0,len(test_df)):
            try:
                if (len(test_df.loc[i,'代號']) != 4):
                    test_df.drop(i,inplace=True)
            except TypeError:
                if test_df.loc[i,'代號'] < 1200 and test_df.loc[i,'代號'] > 9999:
                    test_df.drop(i,inplace=True)
        try:
            test_df = test_df[['代號', '三大法人買賣超股數合計']]
        except KeyError:
            test_df = test_df[['代號', '三大法人買賣超股數']]
        test_df.reset_index(drop=True,inplace=False)
    else:
        del test_df
    return test_df

def crawler_Yield(datetime):
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/peratio_analysis/pera_result.php?l=zh-tw&o=csv&charset=UTF-8&d='+datetime+'&c=&s=0,asc'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
            if '股票代號' in text:
                initial_point = i
                break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
    if (len(use_text) < 10):
        del test_df
    else:
        test_df = test_df[['股票代號', '殖利率(%)']]
        test_df = test_df.rename(columns={"股票代號": '代號'})
    return test_df

#Day_TPEx = []
def trans_date(datetime):
#    global Day_TPEx
    d = datetime.day
    m = datetime.month
    y = datetime.year-1911
    if m < 10:
        if d < 10:
            Day_TPEx = (str(y)+'/0'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/0'+str(m)+'/'+str(d))
    else:
        if d < 10:
            Day_TPEx = (str(y)+'/'+str(m)+'/0'+str(d))
        else:
            Day_TPEx = (str(y)+'/'+str(m)+'/'+str(d))
            
    return Day_TPEx

Crawler_Price = pd.DataFrame()
Crawler_Foundation = pd.DataFrame()
Crawler_Yield = pd.DataFrame()
def crawler_data(datetime):
    try:
        global Crawler_Price
        global Crawler_Foundation
        global Crawler_Yield
        Crawler_Price = crawler(trans_date(datetime))
        Crawler_Foundation = crawler_Foundation(trans_date(datetime))
        Crawler_Yield = crawler_Yield(trans_date(datetime))
        print('TPEx Crawler Successful')
    except UnboundLocalError:
        print('TPEx Crawler Fail')
    except KeyError:
        print('TPEx Crawler Fail')

In [8]:
def catch(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX?date='+datetime+'&type=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['tables'][8]['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['tables'][8]['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '收盤價', '成交股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0],temp.columns[2][0]])
    return test_df

def catch_Foundation(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/fund/T86?date='+datetime+'&selectType=ALLBUT0999&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    for i in range(0,len(test_df)):
        if ((len(test_df.loc[i,'證券代號'][0])) != 4):
            test_df.drop(i,inplace=True)
        elif int(test_df.loc[i,'證券代號'][0]) < 1100 or int(test_df.loc[i,'證券代號'][0]) > 9999:
            test_df.drop(i,inplace=True)
    temp = test_df[['證券代號', '三大法人買賣超股數']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0]])
    test_df.sort_values(by=['證券代號'], inplace=True)
    test_df.reset_index(drop=True,inplace=True)
    return test_df

def catch_Yield(datetime):
    url = 'https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date='+datetime+'&selectType=ALL&response=json'
    page = requests.get(url).text
    json_data = json.loads(page)
    Stock_data = json_data['data']
    test_df = pd.DataFrame(Stock_data, columns = [json_data['fields']])
    temp = test_df[['證券代號', '殖利率(%)']]
    test_df = pd.DataFrame(temp.values, columns = [temp.columns[0][0],temp.columns[1][0]])
    return test_df
    
def catch_day(datetime):
    d = datetime.day
    m = datetime.month
    y = datetime.year
    if m < 10:
        if d < 10:
            Day_TWSE = str(y)+'0'+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+'0'+str(m)+str(d)
    else:
        if d < 10:
            Day_TWSE = str(y)+str(m)+'0'+str(d)
        else:
            Day_TWSE = str(y)+str(m)+str(d)
            
    return Day_TWSE

Catch_Price = pd.DataFrame()
Catch_Foundation = pd.DataFrame()
Catch_Yield = pd.DataFrame()
def catch_data(datetime):
    try:
        global Catch_Price
        global Catch_Foundation
        global Catch_Yield
        Catch_Price = catch(catch_day(datetime))
        Catch_Foundation = catch_Foundation(catch_day(datetime))
        Catch_Yield = catch_Yield(catch_day(datetime))
        print('TWSE Crawler Successful')
    except UnboundLocalError:
        print('TWSE Crawler Fail')
    except KeyError:
        print('TWSE Crawler Fail')

In [9]:
Crawler_Frame = pd.DataFrame()
Catch_Frame = pd.DataFrame()
def Get_New_Data(datetime):
    start = 0
    global Crawler_Frame
    global Crawler_Price
    global Crawler_Foundation
    global Crawler_Yield
    global Catch_Frame
    global Catch_Price
    global Catch_Foundation
    global Catch_Yield
    crawler_data(datetime)
    Crawler_Frame = pd.concat([Crawler_Price.set_index('代號'), Crawler_Foundation.set_index('代號').reindex(Crawler_Price.set_index('代號').index), Crawler_Yield.set_index('代號').reindex(Crawler_Price.set_index('代號').index)], axis=1)
    Crawler_Frame = Crawler_Frame.reset_index()
    catch_data(datetime)
    Catch_Frame = pd.concat([Catch_Price.set_index('證券代號'), Catch_Foundation.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index), Catch_Yield.set_index('證券代號').reindex(Catch_Price.set_index('證券代號').index)], axis=1)
    Catch_Frame = Catch_Frame.reset_index()

In [10]:
def Time_transfer(datetime):
  d = datetime.day
  m = datetime.month
  y = datetime.year
  if m < 10:
    if d < 10:
      Time = str(y)+'-0'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-0'+str(m)+'-'+str(d)
  else:
    if d < 10:
      Time = str(y)+'-'+str(m)+'-0'+str(d)
    else:
      Time = str(y)+'-'+str(m)+'-'+str(d)
  return Time

In [11]:
def Update_Stock_Price(datetime):
  temp = 0
  try:
    url = 'https://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&o=csv&d='+trans_date(datetime)+'&se=EW&s=0,asc,0'
    page = requests.get(url)
    use_text = page.text.splitlines()
    for i,text in enumerate(use_text):
      if '代號' in text:
        initial_point = i
        break
    test_df = pd.read_csv(io.StringIO(''.join([text+ '\n' for text in use_text[initial_point:]])))
  except NameError:
    print('No New Trading Data')
    temp = 1
    
#  global different_Tpex
#  global different_TWSE
#  Update_Stock_List()
                            
  if temp == 0:
    Update_Stock_List()
    Get_New_Data(datetime)
    global StockPrice_TWSE
    global StockPrice_TPEx
    temp1 = 0
    temp2 = 0
    if StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,0] == Time_transfer(datetime):
      print('TWSE Exists!')
      temp1 = 1
    else:
      StockPrice_TWSE = StockPrice_TWSE.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    
    if StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,0] == Time_transfer(datetime):
      print('TPEx Exists!')
      temp2 = 1
    else:
      StockPrice_TPEx = StockPrice_TPEx.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
    
    if temp2 == 0:
      for i in range (0,len(Crawler_Frame)):
        for j in range (1, len(list(StockPrice_TPEx)),4):
          if (Crawler_Frame.loc[i,'代號'] == list(StockPrice_TPEx)[j][0]):
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = float(Crawler_Frame.iloc[i,1])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = float(Crawler_Frame.iloc[i,2].replace(',',''))
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = float(Crawler_Frame.iloc[i,3].replace(',',''))
            except ValueError:              
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+2] = None
            except AttributeError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+1] = None
            
            try:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = float(Crawler_Frame.iloc[i,4])
            except ValueError:
              StockPrice_TPEx.iloc[len(StockPrice_TPEx)-1,j+3] = None
    
    if temp1 == 0:
      for x in range (0,len(Catch_Frame)):
        for y in range (1, len(list(StockPrice_TWSE)),4):
          if (Catch_Frame.loc[x,'證券代號'] == list(StockPrice_TWSE)[y][0]):
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = float(Catch_Frame.iloc[x,1])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = float(Catch_Frame.iloc[x,2].replace(',',''))
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = float(Catch_Frame.iloc[x,3].replace(',',''))
            except ValueError:              
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+2] = None
            except AttributeError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+1] = None
            
            try:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = float(Catch_Frame.iloc[x,4])
            except ValueError:
              StockPrice_TWSE.iloc[len(StockPrice_TWSE)-1,y+3] = None
            
    print('Finished!')  

In [64]:
Update_Stock_Price(datetime.datetime(2023, 5, 8, 10, 1, 55, 280233))

TPEx Pass
TWSE Pass
TPEx Crawler Successful
TWSE Crawler Successful


C:\Users\Richie\AppData\Local\Temp\ipykernel_17332\3166547493.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StockPrice_TWSE = StockPrice_TWSE.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)
C:\Users\Richie\AppData\Local\Temp\ipykernel_17332\3166547493.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  StockPrice_TPEx = StockPrice_TPEx.append({('Date',       'Date'):Time_transfer(datetime)}, ignore_index=True)


Finished!


In [65]:
StockPrice_TWSE

Date   1101                                1102             \
            Date  Close      Volume Foundation Yield  Close     Volume   
0     2016-01-04  26.15  10437306.0 -2717840.0  9.52  26.25  6852789.0   
1     2016-01-05  26.40   5382070.0  1819649.0  9.43  26.00  3967822.0   
2     2016-01-06  26.05   5154746.0  -135962.0  9.56  25.95  3736159.0   
3     2016-01-07  27.35  20289500.0  8650064.0  9.10  26.15  5483126.0   
4     2016-01-08  27.00  11915555.0 -3786818.0  9.22  26.00  5233511.0   
...          ...    ...         ...        ...   ...    ...        ...   
1785  2023-05-02  39.90  69164237.0  9252684.0  1.25  44.25  4009660.0   
1786  2023-05-03  39.50  35068167.0 -8957502.0  1.27  44.15  2315869.0   
1787  2023-05-04  39.45  19369711.0  3013804.0  1.27  44.35  3610228.0   
1788  2023-05-05  39.30  15901928.0 -1239822.0  1.27  44.25  2308727.0   
1789  2023-05-08  39.60  26175572.0  1685790.0  1.26  44.50  5476165.0   

                        1103  ...       9946         9955              \
     Foundation Yield  Close  ... Foundation Yield  Close      Volume   
0    -2582600.0  8.38   9.40  ...    11000.0  3.09  12.15     77112.0   
1     -390654.0  8.46   9.35  ...   -20000.0  3.13  12.20     74106.0   
2      -27353.0  8.48   9.27  ...   -39000.0  3.20  12.10     25100.0   
3     -142190.0  8.41   9.26  ...   -47365.0  3.42  12.20    124337.0   
4    -2929390.0  8.46   9.23  ...     2000.0  3.43  12.50    207000.0   
...         ...   ...    ...  ...        ...   ...    ...         ...   
1785  1018649.0  5.20  20.20  ...    20000.0  2.97  26.25   1390188.0   
1786  -419738.0  5.21  20.20  ...    13000.0  2.97  26.55   4708578.0   
1787   631909.0  5.19  20.20  ...    -5000.0  2.97  26.90   6947792.0   
1788  -812306.0  5.20  20.15  ...    17000.0  2.97  29.55  15617396.0   
1789  -281333.0  5.17  20.35  ...    19000.0  2.99  26.90  10503408.0   

                         9958                              
     Foundation Yield   Close     Volume Foundation Yield  
0        4000.0   0.0    9.95    56147.0    -7000.0  3.02  
1      -19000.0   0.0    9.95   101434.0   -20000.0  3.02  
2           NaN   0.0    9.84    71995.0        NaN  3.05  
3        2337.0   0.0    9.78   112830.0    -7000.0  3.07  
4      -23000.0   0.0    9.86   101907.0    39000.0  3.04  
...         ...   ...     ...        ...        ...   ...  
1785  -319000.0   0.0  113.50  2415222.0  -439479.0  0.44  
1786  -597000.0   0.0  116.00  8063123.0   838000.0  0.43  
1787   132000.0   0.0  117.50  3965003.0   669097.0  0.43  
1788  4284000.0   0.0  114.00  2965814.0 -1168573.0  0.44  
1789 -3417000.0   0.0  109.00  8149416.0    88326.0  0.46  

[1790 rows x 3901 columns]

In [45]:
StockPrice_TPEx

Date   1240                             1258                       \
            Date  Close   Volume Foundation Yield  Close    Volume Foundation   
0     2016-01-04    NaN      NaN        NaN   NaN  39.50   51000.0        NaN   
1     2016-01-05    NaN      NaN        NaN   NaN  38.90  116000.0        NaN   
2     2016-01-06    NaN      NaN        NaN   NaN  38.05   85000.0        NaN   
3     2016-01-07    NaN      NaN        NaN   NaN  36.50  109000.0        NaN   
4     2016-01-08    NaN      NaN        NaN   NaN  37.30   59000.0        NaN   
...          ...    ...      ...        ...   ...    ...       ...        ...   
1777  2023-04-19  46.60  54000.0        0.0  2.58  26.80   16000.0     3000.0   
1778  2023-04-20  46.20  20000.0    -3000.0  2.60  26.80   29000.0     8000.0   
1779  2023-04-21  46.15  33000.0    -5026.0  2.60  26.80       NaN        NaN   
1780  2023-04-24  46.00  14000.0    -1000.0  2.61  26.80   10000.0    -2000.0   
1781  2023-04-25  45.00  28000.0    -1000.0  2.67  26.80       NaN        NaN   

            1259  ...       9951         9960                            \
     Yield Close  ... Foundation Yield  Close   Volume Foundation Yield   
0     3.29  72.1  ...        NaN  3.33  19.00   2000.0        NaN  4.21   
1     3.34  70.9  ...        NaN  3.42  18.90   4000.0        NaN  4.23   
2     3.42  70.3  ...        NaN  3.48  18.70   2000.0        NaN  4.28   
3     3.56  69.1  ...        NaN  3.54  18.85  78000.0        NaN  4.24   
4     3.49  71.8  ...        NaN  3.41  18.75  83000.0        NaN  4.27   
...    ...   ...  ...        ...   ...    ...      ...        ...   ...   
1777  0.00  74.1  ...    24000.0  4.55  30.20  53000.0    -1000.0  7.28   
1778  0.00  74.1  ...   -19000.0  4.59  29.95      NaN        NaN  7.35   
1779  0.00  74.1  ...   -73000.0  4.73  29.75  34000.0    -3000.0  7.39   
1780  0.00  73.5  ...   -18000.0  4.67  29.80      NaN        NaN  7.38   
1781  0.00  72.6  ...   -24000.0  4.76  29.70      NaN        NaN  7.41   

       9962                              
      Close     Volume Foundation Yield  
0      7.25     1000.0        NaN  2.76  
1      7.17     2000.0        NaN  2.79  
2      7.06    35000.0        NaN  2.83  
3      7.07    30000.0        NaN  2.83  
4      7.06     1000.0        NaN  2.83  
...     ...        ...        ...   ...  
1777  20.85  1406000.0  -279000.0  8.63  
1778  20.30  1363000.0  -365000.0  8.87  
1779  19.70  1279000.0   -92000.0  9.14  
1780  19.95   647000.0    45000.0  9.02  
1781  19.25  1190000.0  -364000.0  9.35  

[1782 rows x 3249 columns]

In [16]:
StockPrice_TWSE.to_csv('Stock_TWSE.csv')
StockPrice_TPEx.to_csv('Stock_TPEx.csv')

In [ ]:
MA_TWSE = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TWSE.columns.get_level_values(0)[1:].drop_duplicates())
MA_TWSE.index.name ='Moving Average'
MA_TPEx = pd.DataFrame(index=['7MA', '14MA', '30MA', '60MA', '90MA', '180MA', '360MA', '720MA', '1080MA'],columns=StockPrice_TPEx.columns.get_level_values(0)[1:].drop_duplicates())
MA_TPEx.index.name ='Moving Average'
for i in range(0,len(MA_TWSE.columns)):
    for j in range(0,len(MA_TWSE)):
        MA_TWSE.iloc[j,i] = round(((StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TWSE.iloc[int('-' + MA_TWSE.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)
    
for i in range(0,len(MA_TPEx.columns)):
    for j in range(0,len(MA_TPEx)):
        MA_TPEx.iloc[j,i] = round(((StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+1] * StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2])/(StockPrice_TPEx.iloc[int('-' + MA_TPEx.index.str.extract('(\d+)',expand=False)[j]):,i*4+2].mean())).mean(), 2)

In [ ]:
MA_TWSE

In [ ]:
MA_TPEx

In [ ]:
temp = [['<7MA', '<14MA', '<30MA', '<60MA', '<90MA', '<180MA', '<360MA', '<720MA', '<1080MA'], ['>3', '>6', '>9']]
Result_TWSE = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
Result_TPEx = pd.DataFrame(columns=pd.MultiIndex.from_product(temp))
def Judgment(condition):
  global Result_TWSE
  global Result_TPEx
  if (condition == '7MA'):
    x = 0
  elif (condition == '14MA'):
    x = 1
  elif (condition == '30MA'):
    x = 2
  elif (condition == '60MA'):
    x = 3
  elif (condition == '90MA'):
    x = 4
  elif (condition == '180MA'):
    x = 5
  elif (condition == '360MA'):
    x = 6
  elif (condition == '720MA'):
    x = 7
  elif (condition == '1080MA'):
    x = 8
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TWSE))):
    if (MA_TWSE.iloc[x,i] > StockPrice_TWSE.iloc[-1,i*4+1]):
        if ((StockPrice_TWSE.iloc[-1,i*4+4] >= 3) and (StockPrice_TWSE.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 6) and (StockPrice_TWSE.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TWSE.columns[i])
        elif ((StockPrice_TWSE.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TWSE.columns[i])
  Result_TWSE.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TWSE.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TWSE.iloc[:,x*3+2] = pd.Series(Pass_2)
  
  Pass_0 = []
  Pass_1 = []
  Pass_2 = []
  for i in range(0,len(list(MA_TPEx))):
    if (MA_TPEx.iloc[x,i] > StockPrice_TPEx.iloc[-1,i*4+1]):
        if ((StockPrice_TPEx.iloc[-1,i*4+4] >= 3) and (StockPrice_TPEx.iloc[-1,i*4+4] < 6)):
            Pass_0.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 6) and (StockPrice_TPEx.iloc[-1,i*4+4] < 9)):
            Pass_1.append(MA_TPEx.columns[i])
        elif ((StockPrice_TPEx.iloc[-1,i*4+4] >= 9)):
            Pass_2.append(MA_TPEx.columns[i])
  Result_TPEx.iloc[:,x*3] = pd.Series(Pass_0)
  Result_TPEx.iloc[:,x*3+1] = pd.Series(Pass_1)
  Result_TPEx.iloc[:,x*3+2] = pd.Series(Pass_2)

In [ ]:
for i in range(0,len(MA_TWSE.index)):
    Judgment(MA_TWSE.index[i])
for i in range(0,len(MA_TPEx.index)):
    Judgment(MA_TPEx.index[i])
#Result_TWSE = Result_TWSE.reindex(sorted(Result_TWSE.columns), axis=1)
#Result_TPEx = Result_TPEx.reindex(sorted(Result_TPEx.columns), axis=1)

In [ ]:
Result_TWSE

In [ ]:
Result_TPEx

In [ ]:
Result_TWSE.to_excel('Result_TWSE.xlsx')
Result_TPEx.to_excel('Result_TPEx.xlsx')

In [ ]:
def highlight(series):
    return ['color: red' if value > (series.mean()*3)
            else 'color: orange' if value > (series.mean()*2)
            else None for value in series]

In [ ]:
Stock_TWSE_recentvolume = StockPrice_TWSE.xs('Volume', axis = 1, level = 1)[-14:]
columns_volume = list(Stock_TWSE_recentvolume.columns)
Stock_TWSE_recentvolume.style.apply(highlight, subset=columns_volume)